In [10]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from data.transaction_data_generator import generate_transaction_data
from typing import List, Set, Dict, Tuple, Optional
from preparation.data_preparer import prepare_data
import plotly.express as px
from visualization.data_visualizer import visualize_data


In [11]:
transactions = generate_transaction_data()

In [12]:
prepare_data(transactions)

In [13]:
year = 2019

In [14]:
selected_year = transactions[transactions['year']==year].copy()

In [15]:
grouped = selected_year.groupby(['month','type']).sum().reset_index()
grouped_week = selected_year.groupby(["week"]).sum().reset_index()


In [16]:
balance = selected_year.groupby(['month']).sum().reset_index()
balance['balance'] =  balance['Credit'] + balance['Debit']
balance["color"] = np.where(balance["balance"]<0, 'Negative', 'Positive')
balance['savings'] = balance['balance'].cumsum()

In [17]:
selected_year ['expense'] = abs(transactions['Debit'])

In [18]:
fig = px.pie(selected_year, values='expense', names='type', title=f'Ratio of expenses by type in year {year}',hole=.3)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()


fig = px.bar(balance, x="month", y="balance",
             color='color',
             barmode='stack',
             labels={
                     "color": "Balance",
                     "balance": "Balance [EUR]",
                     "month": "Month"
             },
             title = f"Monthly balance in year {year}",
             height=400)

fig.show()
fig = px.bar(grouped, x="month", y="Debit",
             labels={
                     "Debit": "Total expenses [EUR]",
                     "month": "Month"
             },
             title = f"Total Expenses by month in year {year}",
             barmode='group',
             height=400)
fig.show()

fig = px.bar(grouped, x="month", y="Debit",
             color='type', barmode='group',
             labels={
                     "type": "Transaction type",
                     "month": "Month",
                     "Debit" : "Expenses [EUR]"
             },
             title = f"Total Expenses by month and type in year {year}",
             height=400)
fig.show()


fig = px.bar(grouped_week, x="week", y="Debit",labels={
                     "savings": "Expenses [EUR]",
                     "week": "Week"
             },
             color_discrete_sequence=[ "green"],
             title=f'Expenses by calendar week for year {year}')
fig.show()


fig = px.area(balance, x="month", y="savings",labels={
                     "savings": "Savings [EUR]",
                     "month": "Month"
             },
             color_discrete_sequence=[ "green"],
             title=f'Savings trend for year {year}')
fig.show()
